In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
import copy
warnings.simplefilter(action='ignore')

In [2]:
data_train_final = pd.read_csv('data_train_final.csv')
data_test_final = pd.read_csv('data_test_final.csv')

In [4]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Embedding, Input, Dense
from tensorflow.keras import layers
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import MultiHeadAttention, LayerNormalization, Dense
import keras_nlp

In [10]:
# Prepare baskets
def prepare_baskets(data):
    return data.groupby("order_id")["product_id"].apply(list).tolist()

train_baskets = prepare_baskets(data_train_final)
test_baskets = prepare_baskets(data_test_final)

train_baskets, val_baskets = train_test_split(train_baskets, test_size=0.25, random_state=42)

max_len = max(len(basket) for basket in train_baskets + val_baskets + test_baskets)

In [11]:
D = 32
batch_size = 256
max_epochs = 1000
lr = 1e-4
max_items = len(set(data_train_final['product_id']))

In [12]:
def preprocess_baskets(baskets):
    context_inputs = []
    target_inputs = []
    masked_idxs = []

    for basket in baskets:
        for idx, elt in enumerate(basket):
            target_inputs.append(elt)
            context_inputs.append(basket[:idx] + [max_items + 1] + basket[(idx+1):])
            masked_idxs.append(idx)

    context_inputs = pad_sequences(context_inputs, padding='post', maxlen = max_len, value=0)
    return np.array(context_inputs), np.array(target_inputs) - 1, np.array(masked_idxs)

train_context_input, train_target_input, train_masked_idxs = preprocess_baskets(train_baskets)
val_context_input, val_target_input, val_masked_idxs = preprocess_baskets(val_baskets)
test_context_input, test_target_input, test_masked_idxs = preprocess_baskets(test_baskets)

In [13]:
train_context_input[2]

array([55, 54, 64, 20,  4,  1, 29, 21, 11,  8, 23,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0], dtype=int32)

In [14]:
train_target_input[2]

11

In [15]:
input_context = layers.Input(shape=(max_len,), dtype=tf.int32, name="context_input")
masked_idx_input = layers.Input(shape=(1,), dtype=tf.int32, name="masked_idx_input")

alpha_embedding = layers.Embedding(input_dim=max_items + 2, output_dim=D, name="alpha_embedding")
context_embedding = alpha_embedding(input_context) 

class ZeroMaskEmbedding(layers.Layer):
    def call(self, embeddings, input_tokens):
        mask = tf.cast(tf.not_equal(input_tokens, 0), tf.float32) 
        mask = tf.expand_dims(mask, axis=-1) 
        return embeddings * mask 

context_embedding = ZeroMaskEmbedding()(context_embedding, input_context)

class MaskLayer(layers.Layer):
    def call(self, input_context, position):
        return position * tf.expand_dims(tf.cast(tf.not_equal(input_context, 0), tf.float32), axis = -1)

position = keras_nlp.layers.PositionEmbedding(sequence_length=max_len)(context_embedding)
masked_position = MaskLayer()(input_context, position)
context_embedding = context_embedding + masked_position

class CreateAttentionMask(layers.Layer):
    def call(self, inputs):
        input_context = inputs
        temp = tf.cast(tf.not_equal(input_context, 0), dtype=tf.float32)
        return tf.expand_dims(tf.expand_dims(temp, axis=1), axis=1)

attention_mask = CreateAttentionMask()(input_context)

attention_layer_1 = MultiHeadAttention(num_heads=2, key_dim=16, name="multi_head_attention_1")
attn_output_1 = attention_layer_1(
    query=context_embedding,
    value=context_embedding,
    key=context_embedding,
    attention_mask=attention_mask
)

attn_output_1 = context_embedding + attn_output_1

attention_layer_2 = MultiHeadAttention(num_heads=2, key_dim=16, name="multi_head_attention_2")
attn_output_2 = attention_layer_2(
    query=attn_output_1,
    value=attn_output_1,
    key=attn_output_1,
    attention_mask=attention_mask
)

context_embedding = attn_output_1 + attn_output_2

class GatherLayer(layers.Layer):
    def call(self, inputs):
        context_embedding, masked_idx_input = inputs
        return tf.gather(context_embedding, indices=tf.squeeze(masked_idx_input, axis=-1), batch_dims=1)

masked_embeddings = GatherLayer()([context_embedding, masked_idx_input])

output = layers.Dense(max_items, activation="softmax", name="output_layer", use_bias = False)(masked_embeddings)

model = Model(inputs=[input_context, masked_idx_input], outputs=output)
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr), loss="sparse_categorical_crossentropy")

early_stopping = EarlyStopping(monitor='val_loss', patience=1000, restore_best_weights=True)

history = model.fit(
    [train_context_input, train_masked_idxs], train_target_input,
    validation_data=([val_context_input, val_masked_idxs], val_target_input),
    batch_size=batch_size,
    epochs=max_epochs,
    callbacks=[early_stopping]
)

Epoch 1/1000
807/807 ━━━━━━━━━━━━━━━━━━━━ 23s 25ms/step - loss: 4.0918 - val_loss: 3.9893
Epoch 2/1000
807/807 ━━━━━━━━━━━━━━━━━━━━ 21s 26ms/step - loss: 3.9883 - val_loss: 3.9595
Epoch 3/1000
807/807 ━━━━━━━━━━━━━━━━━━━━ 21s 26ms/step - loss: 3.9490 - val_loss: 3.8998
Epoch 4/1000
807/807 ━━━━━━━━━━━━━━━━━━━━ 20s 25ms/step - loss: 3.8954 - val_loss: 3.8706
Epoch 5/1000
807/807 ━━━━━━━━━━━━━━━━━━━━ 19s 24ms/step - loss: 3.8702 - val_loss: 3.8543
Epoch 6/1000
807/807 ━━━━━━━━━━━━━━━━━━━━ 20s 24ms/step - loss: 3.8547 - val_loss: 3.8423
Epoch 7/1000
807/807 ━━━━━━━━━━━━━━━━━━━━ 20s 23ms/step - loss: 3.8439 - val_loss: 3.8300
Epoch 8/1000
807/807 ━━━━━━━━━━━━━━━━━━━━ 19s 24ms/step - loss: 3.8301 - val_loss: 3.8126
Epoch 9/1000
807/807 ━━━━━━━━━━━━━━━━━━━━ 21s 26ms/step - loss: 3.8073 - val_loss: 3.7902
Epoch 10/1000
807/807 ━━━━━━━━━━━━━━━━━━━━ 19s 23ms/step - loss: 3.7873 - val_loss: 3.7707
Epoch 11/1000
807/807 ━━━━━━━━━━━━━━━━━━━━ 21s 26ms/step - loss: 3.7640 - val_loss: 3.7551
Epoch 12

807/807 ━━━━━━━━━━━━━━━━━━━━ 16s 19ms/step - loss: 3.5171 - val_loss: 3.5261
Epoch 132/1000
807/807 ━━━━━━━━━━━━━━━━━━━━ 16s 19ms/step - loss: 3.5160 - val_loss: 3.5266
Epoch 133/1000
807/807 ━━━━━━━━━━━━━━━━━━━━ 16s 19ms/step - loss: 3.5165 - val_loss: 3.5249
Epoch 134/1000
807/807 ━━━━━━━━━━━━━━━━━━━━ 16s 19ms/step - loss: 3.5098 - val_loss: 3.5232
Epoch 135/1000
807/807 ━━━━━━━━━━━━━━━━━━━━ 16s 19ms/step - loss: 3.5127 - val_loss: 3.5225
Epoch 136/1000
807/807 ━━━━━━━━━━━━━━━━━━━━ 16s 19ms/step - loss: 3.5092 - val_loss: 3.5224
Epoch 137/1000
807/807 ━━━━━━━━━━━━━━━━━━━━ 16s 20ms/step - loss: 3.5139 - val_loss: 3.5206
Epoch 138/1000
807/807 ━━━━━━━━━━━━━━━━━━━━ 16s 19ms/step - loss: 3.5084 - val_loss: 3.5197
Epoch 139/1000
807/807 ━━━━━━━━━━━━━━━━━━━━ 15s 19ms/step - loss: 3.5064 - val_loss: 3.5184
Epoch 140/1000
807/807 ━━━━━━━━━━━━━━━━━━━━ 15s 19ms/step - loss: 3.5033 - val_loss: 3.5204
Epoch 141/1000
807/807 ━━━━━━━━━━━━━━━━━━━━ 15s 19ms/step - loss: 3.5079 - val_loss: 3.5165
Epo

807/807 ━━━━━━━━━━━━━━━━━━━━ 18s 22ms/step - loss: 3.4524 - val_loss: 3.4698
Epoch 262/1000
807/807 ━━━━━━━━━━━━━━━━━━━━ 17s 21ms/step - loss: 3.4460 - val_loss: 3.4693
Epoch 263/1000
807/807 ━━━━━━━━━━━━━━━━━━━━ 16s 20ms/step - loss: 3.4474 - val_loss: 3.4696
Epoch 264/1000
807/807 ━━━━━━━━━━━━━━━━━━━━ 21s 20ms/step - loss: 3.4520 - val_loss: 3.4683
Epoch 265/1000
807/807 ━━━━━━━━━━━━━━━━━━━━ 16s 20ms/step - loss: 3.4456 - val_loss: 3.4685
Epoch 266/1000
807/807 ━━━━━━━━━━━━━━━━━━━━ 16s 20ms/step - loss: 3.4527 - val_loss: 3.4675
Epoch 267/1000
807/807 ━━━━━━━━━━━━━━━━━━━━ 16s 20ms/step - loss: 3.4413 - val_loss: 3.4674
Epoch 268/1000
807/807 ━━━━━━━━━━━━━━━━━━━━ 16s 20ms/step - loss: 3.4403 - val_loss: 3.4672
Epoch 269/1000
807/807 ━━━━━━━━━━━━━━━━━━━━ 16s 20ms/step - loss: 3.4479 - val_loss: 3.4661
Epoch 270/1000
807/807 ━━━━━━━━━━━━━━━━━━━━ 20s 20ms/step - loss: 3.4461 - val_loss: 3.4664
Epoch 271/1000
807/807 ━━━━━━━━━━━━━━━━━━━━ 16s 20ms/step - loss: 3.4432 - val_loss: 3.4660
Epo

807/807 ━━━━━━━━━━━━━━━━━━━━ 20s 24ms/step - loss: 3.4172 - val_loss: 3.4435
Epoch 392/1000
807/807 ━━━━━━━━━━━━━━━━━━━━ 21s 27ms/step - loss: 3.4107 - val_loss: 3.4432
Epoch 393/1000
807/807 ━━━━━━━━━━━━━━━━━━━━ 20s 25ms/step - loss: 3.4144 - val_loss: 3.4434
Epoch 394/1000
807/807 ━━━━━━━━━━━━━━━━━━━━ 20s 25ms/step - loss: 3.4118 - val_loss: 3.4432
Epoch 395/1000
807/807 ━━━━━━━━━━━━━━━━━━━━ 20s 24ms/step - loss: 3.4127 - val_loss: 3.4425
Epoch 396/1000
807/807 ━━━━━━━━━━━━━━━━━━━━ 21s 26ms/step - loss: 3.4140 - val_loss: 3.4422
Epoch 397/1000
807/807 ━━━━━━━━━━━━━━━━━━━━ 21s 25ms/step - loss: 3.4137 - val_loss: 3.4425
Epoch 398/1000
807/807 ━━━━━━━━━━━━━━━━━━━━ 20s 25ms/step - loss: 3.4084 - val_loss: 3.4422
Epoch 399/1000
807/807 ━━━━━━━━━━━━━━━━━━━━ 20s 24ms/step - loss: 3.4148 - val_loss: 3.4418
Epoch 400/1000
807/807 ━━━━━━━━━━━━━━━━━━━━ 21s 25ms/step - loss: 3.4152 - val_loss: 3.4421
Epoch 401/1000
807/807 ━━━━━━━━━━━━━━━━━━━━ 19s 24ms/step - loss: 3.4140 - val_loss: 3.4421
Epo

807/807 ━━━━━━━━━━━━━━━━━━━━ 17s 21ms/step - loss: 3.3932 - val_loss: 3.4345
Epoch 522/1000
807/807 ━━━━━━━━━━━━━━━━━━━━ 18s 22ms/step - loss: 3.3922 - val_loss: 3.4344
Epoch 523/1000
807/807 ━━━━━━━━━━━━━━━━━━━━ 19s 20ms/step - loss: 3.3920 - val_loss: 3.4339
Epoch 524/1000
807/807 ━━━━━━━━━━━━━━━━━━━━ 16s 20ms/step - loss: 3.3970 - val_loss: 3.4343
Epoch 525/1000
807/807 ━━━━━━━━━━━━━━━━━━━━ 17s 20ms/step - loss: 3.3948 - val_loss: 3.4341
Epoch 526/1000
807/807 ━━━━━━━━━━━━━━━━━━━━ 17s 21ms/step - loss: 3.3865 - val_loss: 3.4336
Epoch 527/1000
807/807 ━━━━━━━━━━━━━━━━━━━━ 17s 20ms/step - loss: 3.3945 - val_loss: 3.4341
Epoch 528/1000
807/807 ━━━━━━━━━━━━━━━━━━━━ 20s 20ms/step - loss: 3.3933 - val_loss: 3.4334
Epoch 529/1000
807/807 ━━━━━━━━━━━━━━━━━━━━ 17s 21ms/step - loss: 3.3956 - val_loss: 3.4335
Epoch 530/1000
807/807 ━━━━━━━━━━━━━━━━━━━━ 20s 20ms/step - loss: 3.3933 - val_loss: 3.4335
Epoch 531/1000
807/807 ━━━━━━━━━━━━━━━━━━━━ 20s 20ms/step - loss: 3.3941 - val_loss: 3.4334
Epo

807/807 ━━━━━━━━━━━━━━━━━━━━ 17s 20ms/step - loss: 3.3837 - val_loss: 3.4297
Epoch 652/1000
807/807 ━━━━━━━━━━━━━━━━━━━━ 16s 20ms/step - loss: 3.3854 - val_loss: 3.4301
Epoch 653/1000
807/807 ━━━━━━━━━━━━━━━━━━━━ 16s 20ms/step - loss: 3.3828 - val_loss: 3.4300
Epoch 654/1000
807/807 ━━━━━━━━━━━━━━━━━━━━ 17s 21ms/step - loss: 3.3848 - val_loss: 3.4304
Epoch 655/1000
807/807 ━━━━━━━━━━━━━━━━━━━━ 17s 21ms/step - loss: 3.3779 - val_loss: 3.4296
Epoch 656/1000
807/807 ━━━━━━━━━━━━━━━━━━━━ 20s 20ms/step - loss: 3.3857 - val_loss: 3.4302
Epoch 657/1000
807/807 ━━━━━━━━━━━━━━━━━━━━ 20s 20ms/step - loss: 3.3788 - val_loss: 3.4294
Epoch 658/1000
807/807 ━━━━━━━━━━━━━━━━━━━━ 17s 20ms/step - loss: 3.3815 - val_loss: 3.4297
Epoch 659/1000
807/807 ━━━━━━━━━━━━━━━━━━━━ 22s 22ms/step - loss: 3.3826 - val_loss: 3.4300
Epoch 660/1000
807/807 ━━━━━━━━━━━━━━━━━━━━ 17s 22ms/step - loss: 3.3767 - val_loss: 3.4304
Epoch 661/1000
807/807 ━━━━━━━━━━━━━━━━━━━━ 16s 20ms/step - loss: 3.3856 - val_loss: 3.4302
Epo

807/807 ━━━━━━━━━━━━━━━━━━━━ 20s 20ms/step - loss: 3.3754 - val_loss: 3.4295
Epoch 782/1000
807/807 ━━━━━━━━━━━━━━━━━━━━ 16s 20ms/step - loss: 3.3700 - val_loss: 3.4291
Epoch 783/1000
807/807 ━━━━━━━━━━━━━━━━━━━━ 16s 20ms/step - loss: 3.3708 - val_loss: 3.4293
Epoch 784/1000
807/807 ━━━━━━━━━━━━━━━━━━━━ 21s 20ms/step - loss: 3.3701 - val_loss: 3.4289
Epoch 785/1000
807/807 ━━━━━━━━━━━━━━━━━━━━ 16s 20ms/step - loss: 3.3722 - val_loss: 3.4303
Epoch 786/1000
807/807 ━━━━━━━━━━━━━━━━━━━━ 16s 20ms/step - loss: 3.3746 - val_loss: 3.4292
Epoch 787/1000
807/807 ━━━━━━━━━━━━━━━━━━━━ 16s 20ms/step - loss: 3.3666 - val_loss: 3.4291
Epoch 788/1000
807/807 ━━━━━━━━━━━━━━━━━━━━ 20s 20ms/step - loss: 3.3674 - val_loss: 3.4286
Epoch 789/1000
807/807 ━━━━━━━━━━━━━━━━━━━━ 17s 21ms/step - loss: 3.3710 - val_loss: 3.4294
Epoch 790/1000
807/807 ━━━━━━━━━━━━━━━━━━━━ 17s 20ms/step - loss: 3.3696 - val_loss: 3.4296
Epoch 791/1000
807/807 ━━━━━━━━━━━━━━━━━━━━ 17s 21ms/step - loss: 3.3704 - val_loss: 3.4290
Epo

807/807 ━━━━━━━━━━━━━━━━━━━━ 16s 20ms/step - loss: 3.3674 - val_loss: 3.4307
Epoch 912/1000
807/807 ━━━━━━━━━━━━━━━━━━━━ 16s 20ms/step - loss: 3.3636 - val_loss: 3.4302
Epoch 913/1000
807/807 ━━━━━━━━━━━━━━━━━━━━ 21s 21ms/step - loss: 3.3628 - val_loss: 3.4299
Epoch 914/1000
807/807 ━━━━━━━━━━━━━━━━━━━━ 17s 20ms/step - loss: 3.3660 - val_loss: 3.4307
Epoch 915/1000
807/807 ━━━━━━━━━━━━━━━━━━━━ 17s 20ms/step - loss: 3.3678 - val_loss: 3.4300
Epoch 916/1000
807/807 ━━━━━━━━━━━━━━━━━━━━ 17s 20ms/step - loss: 3.3604 - val_loss: 3.4298
Epoch 917/1000
807/807 ━━━━━━━━━━━━━━━━━━━━ 20s 20ms/step - loss: 3.3632 - val_loss: 3.4307
Epoch 918/1000
807/807 ━━━━━━━━━━━━━━━━━━━━ 17s 21ms/step - loss: 3.3612 - val_loss: 3.4299
Epoch 919/1000
807/807 ━━━━━━━━━━━━━━━━━━━━ 18s 22ms/step - loss: 3.3671 - val_loss: 3.4301
Epoch 920/1000
807/807 ━━━━━━━━━━━━━━━━━━━━ 19s 21ms/step - loss: 3.3616 - val_loss: 3.4302
Epoch 921/1000
807/807 ━━━━━━━━━━━━━━━━━━━━ 20s 20ms/step - loss: 3.3642 - val_loss: 3.4298
Epo

In [16]:
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ context_input       │ (None, 24)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ alpha_embedding     │ (None, 24, 32)    │      2,080 │ context_input[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ zero_mask_embedding │ (None, 24, 32)    │          0 │ alpha_embedding[… │
│ (ZeroMaskEmbedding) │                   │            │ context_input[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ position_embedding  │ (None, 24, 32)    │        768 │ zero_mask_embedd… │
│ (PositionEmbedding) │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ mask_layer          │ (None, 24, 32)    │          0 │ context_input[0]… │
│ (MaskLayer)         │                   │            │ position_embeddi… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add (Add)           │ (None, 24, 32)    │          0 │ zero_mask_embedd… │
│                     │                   │            │ mask_layer[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ create_attention_m… │ (None, 1, 1, 24)  │          0 │ context_input[0]… │
│ (CreateAttentionMa… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multi_head_attenti… │ (None, 24, 32)    │      4,224 │ create_attention… │
│ (MultiHeadAttentio… │                   │            │ add[0][0],        │
│                     │                   │            │ add[0][0],        │
│                     │                   │            │ add[0][0]         │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_1 (Add)         │ (None, 24, 32)    │          0 │ add[0][0],        │
│                     │                   │            │ multi_head_atten… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multi_head_attenti… │ (None, 24, 32)    │      4,224 │ create_attention… │
│ (MultiHeadAttentio… │                   │            │ add_1[0][0],      │
│                     │                   │            │ add_1[0][0],      │
│                     │                   │            │ add_1[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_2 (Add)         │ (None, 24, 32)    │          0 │ add_1[0][0],      │
│                     │                   │            │ multi_head_atten… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ masked_idx_input    │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ gather_layer        │ (None, 32)        │          0 │ add_2[0][0],      │
│ (GatherLayer)       │                   │            │ masked_idx_input… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ output_layer        │ (None, 63)        │      2,016 │ gather_layer[0][… │
│ (Dense)             │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 39,938 (156.01 KB)

 Trainable params: 13,312 (52.00 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 26,626 (104.01 KB)

In [17]:
# Evaluate on Test Data
test_loss = model.evaluate([test_context_input, test_masked_idxs], test_target_input, batch_size=batch_size)
print(f"Test Loss: {test_loss}")

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 3.4077
Test Loss: 3.4200100898742676


In [18]:
alpha_embedding_layer = model.get_layer("alpha_embedding")
alpha_embedding_weights = alpha_embedding_layer.get_weights()[0][1:-1]

In [19]:
output_layer = model.get_layer("output_layer")
output_layer_weights = output_layer.get_weights()[0]

In [20]:
sim_matrix = pd.DataFrame(np.matmul(alpha_embedding_weights, output_layer_weights) + \
    np.matmul(output_layer_weights.T, alpha_embedding_weights.T))
sim_matrix

,0,1,2,3,4,5,6,7,8,9,...,53,54,55,56,57,58,59,60,61,62
0,-1.278881,0.961786,1.502928,0.181268,0.701960,0.696065,0.478962,-0.081179,0.414173,0.715838,...,0.450187,-0.441200,-0.160474,0.685325,0.544994,-0.249554,1.023250,0.908196,0.046914,-1.965145
1,0.961786,0.179089,-0.539649,-0.557275,0.504567,-0.768874,0.341734,0.784327,0.001474,-0.167526,...,0.726007,-0.376831,0.025756,0.212404,-0.371845,0.899566,0.186164,-0.436419,0.683828,1.123866
2,1.502928,-0.539649,1.444379,-0.592325,0.118927,-0.615966,0.795742,-0.921486,-0.392635,0.005684,...,-0.457851,0.333081,0.056493,0.368688,0.149699,-0.793199,-0.195186,0.090336,-0.311970,0.115448
3,0.181268,-0.557275,-0.592325,-1.789726,-1.097574,-0.130785,-0.539572,-0.459690,-0.270622,0.667493,...,0.634118,0.475764,0.530592,0.317276,0.461045,0.983993,-0.159826,0.761545,0.212753,0.876729
4,0.701960,0.504567,0.118927,-1.097574,0.545602,-0.881793,-0.140245,0.095717,0.083242,-0.697750,...,-0.086640,-0.079916,-0.679038,-0.147210,-0.188495,0.031273,1.260324,-0.997601,-0.295344,-0.441825
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58,-0.249554,0.899566,-0.793199,0.983993,0.031273,0.214799,-0.098214,0.156860,0.085329,-0.332829,...,0.663244,-0.230162,-0.432321,-0.597847,-0.441205,0.666715,-0.855415,-1.196749,-0.555673,-0.926800
59,1.023250,0.186164,-0.195186,-0.159826,1.260324,0.636778,-0.418268,0.539977,-0.113164,0.038206,...,-0.643103,0.270900,0.000010,0.172596,0.066060,-0.855415,-1.598433,0.746913,1.680494,0.081729
60,0.908196,-0.436419,0.090336,0.761545,-0.997601,-0.405659,0.790165,-0.014310,0.340900,0.267681,...,-0.445248,0.447503,-0.654703,0.009825,0.210322,-1.196749,0.746913,0.697369,0.678572,0.380809
61,0.046914,0.683828,-0.311970,0.212753,-0.295344,-0.007181,-0.440952,0.731947,0.899140,0.265888,...,-0.275013,0.405923,-0.572193,0.244660,0.139207,-0.555673,1.680494,0.678572,0.619422,0.610543


In [26]:
for i in range(sim_matrix.shape[0]):
    sim_matrix[i][i] = -1000

In [27]:
top_5_indices_desc = np.argsort(sim_matrix, axis=1)[:, -5:][:, ::-1]

top_5_dict = {}
for i in range(top_5_indices_desc.shape[0]):
    top_5_dict[i] = list(top_5_indices_desc[i])

In [28]:
products = pd.read_csv('products.csv')
products_dict = {}
for i in range(products.shape[0]):
    products_dict[products['product_id'][i]] = products['product_name'][i]

In [29]:
product_ids = [21903, 30391, 46667, 13176, 21616,  8518, 22935,  5876, 48679,
       24838, 31717, 47209, 26209, 34969, 27966, 37646, 44632, 16797,
       39275,  5077, 10749, 49235, 21137, 28204, 21938, 46979, 47626,
       44359, 34126, 28985, 24852, 41950, 30489,  9076, 24964, 45007,
       42265, 49683, 47766, 39877, 19057, 40706,  5450, 43961, 39928,
       22825, 12341, 17794,  4605, 22035, 27845, 27104, 26604,  8277,
        4920, 25890, 31506, 35951, 45066, 24184, 19660, 27086, 43352]

all_products = []
for i in product_ids:
    all_products.append(products_dict[i])

In [30]:
top_5_dict_items = {}

for k, v in top_5_dict.items():
    key = all_products[k]
    value = [all_products[val] for val in v]
    
    top_5_dict_items[key] = value

In [31]:
top_5_dict_items

{'Organic Baby Spinach': ['Organic Small Bunch Celery',
  'Organic Ginger Root',
  'Michigan Organic Kale',
  'Red Peppers',
  'Organic Cucumber'],
 'Organic Cucumber': ['Large Lemon',
  'Organic Zucchini',
  'Organic Fuji Apple',
  'Cucumber Kirby',
  'Raspberries'],
 'Organic Ginger Root': ['Organic Baby Spinach',
  'Organic Cilantro',
  'Organic Yellow Onion',
  'Michigan Organic Kale',
  'Organic Garlic'],
 'Bag of Organic Bananas': ['Organic Cilantro',
  'Organic Fuji Apple',
  'Honeycrisp Apple',
  'Sparkling Water Grapefruit',
  'Organic Large Extra Fancy Fuji Apple'],
 'Organic Baby Arugula': ['Red Peppers',
  'Limes',
  'Organic Red Bell Pepper',
  'Organic Baby Spinach',
  'Asparagus'],
 'Organic Red Onion': ['Limes',
  'Organic Baby Spinach',
  'Organic Avocado',
  'Red Peppers',
  'Carrots'],
 'Organic Yellow Onion': ['Yellow Onions',
  'Green Bell Pepper',
  'Asparagus',
  'Organic Blackberries',
  'Organic Baby Carrots'],
 'Organic Lemon': ['Limes',
  'Organic Garlic',
  